## Машинное обучение — Итоговая аттестация

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pickle

import tensorflow as tf
from keras import layers, models
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

### Загрузка данных

In [3]:
def load_cifar10_data():
    train_data = {}
    for i in range(1, 6):
        with open(f'cifar-10-batches-py/data_batch_{i}', 'rb') as f:
            new_data = pickle.load(f, encoding='bytes')
            train_data = dict(list(train_data.items()) + list(new_data.items()))
    
    X_train = np.array(train_data[b'data'], dtype=np.float32) / 255.0
    y_train = np.array(train_data[b'labels'])
    
    with open(f'cifar-10-batches-py/test_batch', 'rb') as f:
        test_data = pickle.load(f, encoding='bytes')
    
    # Preprocess data
    X_test = np.array(test_data[b'data'], dtype=np.float32) / 255.0  # Normalize pixel values and change data type to float32
    y_test = np.array(test_data[b'labels'])

    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_cifar10_data()
print(X_train)
print(y_train)
print()
print(X_test)
print(y_test)

[[1.         0.9882353  0.99215686 ... 0.6784314  0.90588236 0.972549  ]
 [0.49803922 0.49411765 0.49803922 ... 0.4        0.42352942 0.4392157 ]
 [0.45490196 0.2509804  0.07450981 ... 0.02745098 0.02352941 0.01960784]
 ...
 [0.13725491 0.15686275 0.16470589 ... 0.3019608  0.25882354 0.19607843]
 [0.7411765  0.7294118  0.7254902  ... 0.6627451  0.67058825 0.67058825]
 [0.8980392  0.9254902  0.91764706 ... 0.6784314  0.63529414 0.6313726 ]]
[1 8 5 ... 9 1 1]

[[0.61960787 0.62352943 0.64705884 ... 0.4862745  0.5058824  0.43137255]
 [0.92156863 0.90588236 0.9098039  ... 0.69803923 0.7490196  0.78039217]
 [0.61960787 0.61960787 0.54509807 ... 0.03137255 0.01176471 0.02745098]
 ...
 [0.07843138 0.07450981 0.05882353 ... 0.19607843 0.20784314 0.18431373]
 [0.09803922 0.05882353 0.09019608 ... 0.3137255  0.31764707 0.3137255 ]
 [0.28627452 0.38431373 0.3882353  ... 0.36862746 0.22745098 0.10196079]]
[3 8 8 ... 5 1 7]


### 1. Объясните, какие элементы вашей сети зависят от количества цветов, какие — от количества классов

Элементы сети, зависящие от количества цветов:
1. количество нейронов во входном слое;
2. количество весов;
3. количество связей.

Элементы сети, зависящие от количества классов для датасета CIFAR-10:
1. также количество нейронов в выходном слое;
2. функция активации.

### 2. Обучить модель. Объяснить место в модели каждого слоя, обосновать выбор гиперпараметров

Всего слоя в моей сети четыре:
1. Входной слой - 64 нейрона
2. Скрытые слои - 256 и 64 нейрона
3. Выходной слой - 10 нейронов (совпадает с количеством слоев)

Количество нейронов во всех слоях, кроме выходного, получено эмпирически, изначально взял 1024 (32х32), но обратил внимание, что финальное значение accuracy не менялось при уменьшии количества входных нейронов, а скорость возрастала многократно 

In [4]:
# Обучение модели
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(256, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

# Оценка качества модели
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", test_accuracy)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\DogNellaf\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\DogNellaf\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\DogNellaf\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\DogNellaf\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\DogNellaf\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filter

### 3. Сравнить качество предсказания при обучении на 20 широких классах с предсказаниями при обучении на 100 узких классах, обобщив предсказания по узким меткам до метки их широкого класса в соответствии с таблицей

In [23]:
import tensorflow as tf
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

EPOCH_COUNT = 1

def get_data(label_mode: str):
    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode=label_mode)

    x_train = x_train / 255.0
    x_test = x_test / 255.0
    y_train = to_categorical(y_train, 100)
    y_test = to_categorical(y_test, 100)

    return (x_train, y_train), (x_test, y_test)

def get_params_by_model(label_mode: str, classes_count: int):
    (X_train, y_train), (X_test, y_test) = get_data(label_mode)
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(classes_count, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=EPOCH_COUNT)
    return model.evaluate(X_test, y_test)

accuracy_fine = get_params_by_model('fine', 100)[1]
accuracy_coarse = get_params_by_model('coarse', 20)[1]

print("Accuracy на 100 точных классах:", accuracy_fine)
print("Accuracy на 20 грубых классах:", accuracy_coarse)

if accuracy_fine > accuracy_coarse:
    print("Предсказание на основе 100 точных классов оказалось точнее")
else:
    print("Предсказание на основе 20 грубых классов оказалось точнее")

ValueError: Input 0 of layer "max_pooling2d_27" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 64)